In [1]:
import os
import ssl
import time
import socket
import requests
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from urllib import parse
from tqdm.notebook import tqdm

from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager #!pip install webdriver-manager

# 한글대응
from matplotlib import rc
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
rc('font',family='Malgun Gothic')

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
def create_folder(directory):
    import os
    
    if not os.path.exists(directory):
        os.mkdir(directory)

In [3]:
category_dict = {
    '치킨' : 5,
    '피자/양식' : 6,
    '중국집' : 7,
    '한식' : 8,
    '일식/돈까스' : 9,
    '족발/보쌈' : 10,
    '야식' : 11,
    '분식' : 12,
    '카페/디저트' : 13,
}

In [4]:
for i in category_dict:
    print(i.replace('/',''))

치킨
피자양식
중국집
한식
일식돈까스
족발보쌈
야식
분식
카페디저트


In [5]:
def yogiyo_crawling(store_rank, get_review_num, save_dir, menu='치킨'):
    url = 'https://www.yogiyo.co.kr/mobile/#/'
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url); action = ActionChains(driver); time.sleep(1)

    driver.maximize_window(); time.sleep(1)
    
    category_tag = 'body > div.category-list > div > div:nth-child(' + str(category_dict[menu]) +') > a > div'
    cate = driver.find_element(By.CSS_SELECTOR, category_tag)
    cate_name = cate.text
    print(f'크롤링 메뉴 : [{cate_name}]')
    cate.click()

    # 리뷰 많은 순으로 정렬 -> 이게 알맞지 않은 가게를 보여줌
    sort_btn = driver.find_element(By.CSS_SELECTOR, '#content > div > div.row.restaurant-list-info > div.list-option > div > select')
    sort_btn.click(); time.sleep(1)
    review_many = driver.find_element(By.CSS_SELECTOR,'#content > div > div.row.restaurant-list-info > div.list-option > div > select > option:nth-child(3)')
    review_many.click(); time.sleep(1)
    
    # 중식 4번째 -> 금룡
    store_tag = '#content > div > div:nth-child(5) > div > div > div:nth-child('+str(store_rank)+') > div > table > tbody > tr > td:nth-child(2) > div > div.restaurant-name.ng-binding'
    store = driver.find_element(By.CSS_SELECTOR, store_tag)
    print(f'크롤링 가게 리뷰순 : {store_rank}위 [{store.text}]')
    store.click(); time.sleep(1)

    # 리뷰탭 클릭
    review_btn = driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(2) > a').click()
    time.sleep(1)
    
    # 사진 리뷰만 보기
    review_img = driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div:nth-child(6) > div.review-count > div > div > div > label').click()
    time.sleep(1)
    
    # 스크롤 맨 아래로
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);"); time.sleep(1)  # 브라우저 끝까지 스크롤을 내림
    
    review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
    
    while (review_num < get_review_num):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 브라우저 끝까지 스크롤을 내림
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, '#review > li.list-group-item.btn-more > a').click()
        time.sleep(1)
        review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
        
    # 현재 화면의 html 코드를 가져와서 뷰숲으로 작업하기
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    
    # 리뷰 목록 불러오기
    review_tables = soup.select('#review>li')[1:get_review_num+1]
    
    make_save_folder = save_dir + 'reviews_data'
    create_folder(make_save_folder)

    for n, review in enumerate(review_tables):
        img_tag = review.select('table>tbody>tr>td>div>img')
        image_urls = [each['data-url'] for each in img_tag]

        make_user_folder = make_save_folder+'/'+cate_name.replace('/','')+'reviewer'+str(n+1)
        create_folder(make_user_folder)

        for idx, url in enumerate(image_urls):
            response = requests.get(url)

            if response.status_code == 200:
                image_file = os.path.join(make_user_folder, f'img{idx+1}.jpg')
                with open(image_file, 'wb') as file:
                    file.write(response.content)
                #print('이미지 저장', image_file)
            else:
                print('이미지 저장 실패')
    
    print(f'[{cate_name}] 크롤링 완료')
    driver.quit()

In [6]:
for menu in category_dict:
    yogiyo_crawling(menu=menu, store_rank=4, get_review_num=20, save_dir='../Data_total/')

크롤링 메뉴 : [치킨]
크롤링 가게 리뷰순 : 4위 [멕시카나-안산고잔점]
[치킨] 크롤링 완료
크롤링 메뉴 : [피자/양식]
크롤링 가게 리뷰순 : 4위 [굽네치킨&피자-안산한양대점]
[피자/양식] 크롤링 완료
크롤링 메뉴 : [중국집]
크롤링 가게 리뷰순 : 4위 [손오공마라탕-안산점]
[중국집] 크롤링 완료
크롤링 메뉴 : [한식]
크롤링 가게 리뷰순 : 4위 [떡순튀-안산점]
[한식] 크롤링 완료
크롤링 메뉴 : [일식/돈까스]
크롤링 가게 리뷰순 : 4위 [돈까스튀기는형-본점]
[일식/돈까스] 크롤링 완료
크롤링 메뉴 : [족발/보쌈]
크롤링 가게 리뷰순 : 4위 [제주총각족발보쌈-안산본점]
[족발/보쌈] 크롤링 완료
크롤링 메뉴 : [야식]
크롤링 가게 리뷰순 : 4위 [지코바치킨-안산고잔1호점]
[야식] 크롤링 완료
크롤링 메뉴 : [분식]
크롤링 가게 리뷰순 : 4위 [신할매국물떡뽀끼]
[분식] 크롤링 완료
크롤링 메뉴 : [카페/디저트]
크롤링 가게 리뷰순 : 4위 [봉명동내커피-안산1호점]
[카페/디저트] 크롤링 완료
